In [2]:
#Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import unidecode
import requests
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
def outcome(row):
    """
    Return outcome of match result which is represented by a numeric value

    Parameters:
    row: specified row in the dataframe 
    """
    home = row['Home_Score']
    away = row['Away_Score']

    if home > away:
        return 1
    elif home < away:
        return -1
    else:
        return 0

In [4]:
def match_result(season):
    """
    Return the all match result of the specified season

    Parameters:
    season: int
        Season to get the match result
    """

    #Dictionary contains each season respectively to its id
    id = {
        2021:'10728',2020:'3232',2019:'1889',2018:'1631',2017:'1526',2016:'1467',
        2015:'733',2014:'669',2013:'602',2012:'534',2011:'467',2010:'400',
        2009:'338',2008:'282',2007:'229',2006:'183',2005:'146',2004:'112',
        2003:'84',2002:'63',2001:'47'
    }
    #URL to get the match result
    url = f'https://fbref.com/en/comps/9/{id[season]}/schedule/{season-1}-{season}-Premier-League-Scores-and-Fixtures'
    #Extract the week, weekday, date, home team, away team and match score information
    df = pd.read_html(url)[0][['Wk','Day','Date','Time','Referee','Attendance','Home','Away','Score']]
    #Remove the empty data
    df.dropna(how='all',inplace=True)
    df.reset_index(drop=True,inplace=True)
    #Create new columns of home and, teams score, outcome match and season
    df['Home_Score'] = df['Score'].apply(lambda x: x.split('–')[0])
    df['Away_Score'] = df['Score'].apply(lambda x: x.split('–')[1])
    df['Outcome'] = df.apply(outcome,axis=1)
    df['Season'] = season
    #Drop redundant columns
    df.drop(['Score'],axis=1,inplace=True)

    return df

def match_result_full():
    """
    Return the all match result of season from 2001 to 2021
    """
    
    full_match_results = []
    for season in range(2001,2022):
        full_match_results.append(match_result(season))
        print(f'Got match result of season {season}')
    df = pd.concat(full_match_results).reset_index(drop=True)

    return df

In [5]:
# results = match_result_full()
# results.to_csv('data/match_results.csv',index=False)
# results

Got match result of season 2001
Got match result of season 2002
Got match result of season 2003
Got match result of season 2004
Got match result of season 2005
Got match result of season 2006
Got match result of season 2007
Got match result of season 2008
Got match result of season 2009
Got match result of season 2010
Got match result of season 2011
Got match result of season 2012
Got match result of season 2013
Got match result of season 2014
Got match result of season 2015
Got match result of season 2016
Got match result of season 2017
Got match result of season 2018
Got match result of season 2019
Got match result of season 2020
Got match result of season 2021


,Wk,Day,Date,Time,Referee,Attendance,Home,Away,Home_Score,Away_Score,Outcome,Season
0,1.0,Sat,2000-08-19,NaN,Alan Wiley,NaN,Tottenham,Ipswich Town,3,1,1,2001
1,1.0,Sat,2000-08-19,NaN,Andy D'Urso,NaN,Derby County,Southampton,2,2,0,2001
2,1.0,Sat,2000-08-19,NaN,Mike Riley,NaN,Leicester City,Aston Villa,0,0,0,2001
3,1.0,Sat,2000-08-19,NaN,Graham Barber,NaN,Chelsea,West Ham,4,2,1,2001
4,1.0,Sat,2000-08-19,NaN,Barry Knight,NaN,Coventry City,Middlesbrough,1,3,-1,2001
...,...,...,...,...,...,...,...,...,...,...,...,...
7975,38.0,Sun,2021-05-23,16:00,Martin Atkinson,10000.0,West Ham,Southampton,3,0,1,2021
7976,38.0,Sun,2021-05-23,16:00,Chris Kavanagh,2000.0,Fulham,Newcastle Utd,0,2,-1,2021
7977,38.0,Sun,2021-05-23,16:00,Jonathan Moss,10000.0,Arsenal,Brighton,2,0,1,2021
7978,38.0,Sun,2021-05-23,16:00,Kevin Friend,5000.0,Sheffield Utd,Burnley,1,0,1,2021


In [5]:
def team_stats(season,week,how='full'):
    """
    Return some statistics of teams (number win matchs, number lose matchs, number draw matchs, rank, goal difference) of specified week and season
    
    Parameters:
    season: int
        Season to get data
    week: int
        Match round of season
    how: str
        "full": statistics of teams when playing as home team and away team
        "home": statistics of teams when playing as home team
        "away": statistics of teams when playing as away team
    """

    #Homogenization teams name
    name_mapping = {
        'AFC Bournemouth': 'Bournemouth', 'Arsenal FC': 'Arsenal',
        'Aston Villa': 'Aston Villa', 'Birmingham City': 'Birmingham City',
        'Blackburn Rovers': 'Blackburn', 'Blackpool FC': 'Blackpool',
        'Bolton Wanderers': 'Bolton', 'Bradford City': 'Bradford City',
        'Brighton & Hove Albion': 'Brighton', 'Burnley FC': 'Burnley',
        'Cardiff City': 'Cardiff City', 'Charlton Athletic': 'Charlton Ath',
        'Chelsea FC': 'Chelsea', 'Coventry City': 'Coventry City',
        'Crystal Palace': 'Crystal Palace', 'Derby County': 'Derby County',
        'Everton FC': 'Everton', 'Fulham FC': 'Fulham', 'Huddersfield Town': 'Huddersfield',
        'Hull City': 'Hull City', 'Ipswich Town': 'Ipswich Town',
        'Leeds United': 'Leeds United', 'Leicester City': 'Leicester City',
        'Liverpool FC': 'Liverpool', 'Manchester City': 'Manchester City',
        'Manchester United': 'Manchester Utd', 'Middlesbrough FC': 'Middlesbrough',
        'Newcastle United': 'Newcastle Utd', 'Norwich City': 'Norwich City',
        'Portsmouth FC': 'Portsmouth', 'Queens Park Rangers': 'QPR',
        'Reading FC': 'Reading', 'Sheffield United': 'Sheffield Utd',
        'Southampton FC': 'Southampton', 'Stoke City': 'Stoke City',
        'Sunderland AFC': 'Sunderland', 'Swansea City': 'Swansea City',
        'Tottenham Hotspur': 'Tottenham', 'Watford FC': 'Watford',
        'West Bromwich Albion': 'West Brom', 'West Ham United': 'West Ham',
        'Wigan Athletic': 'Wigan Athletic', 'Wolverhampton Wanderers': 'Wolves'
    }

    #Filtering methods
    if how == 'full':
        url = f'https://www.worldfootball.net/schedule/eng-premier-league-{season-1}-{season}-spieltag/{week}/'
        df = pd.read_html(url)[3][['Team.1','W','D','L','goals','Dif.']]
    elif how == 'home':
        url = f'https://www.worldfootball.net/schedule/eng-premier-league-{season-1}-{season}-spieltag/{week}/heim/'
        df = pd.read_html(url)[2][['Team.1','W','D','L','goals','Dif.']]
    else:
        url = f'https://www.worldfootball.net/schedule/eng-premier-league-{season-1}-{season}-spieltag/{week}/auswaerts/'
        df = pd.read_html(url)[2][['Team.1','W','D','L','goals','Dif.']]

    #Rename columns and create some new columns
    df.columns = ['Team','W','D','L','Goals','Diff']
    df['Goal'] = df['Goals'].apply(lambda x: x.split(':')[0])
    df['Opponent Goal'] = df['Goals'].apply(lambda x: x.split(':')[1])
    df['Rank'] = range(1,21)
    df['Team'] = df['Team'].map(name_mapping)
    df['wk'] = week
    df['Season'] = season

    #Remove redundant columns
    df.drop(['Goals'],axis=1,inplace=True)
    
    return df

def team_stats_full_week(season,how='full'):
    """
    Return some statistics of teams (number win matchs, number lose matchs, number draw matchs, rank, goal difference) in all week of specified season
    
    Parameters:
    season: int
        Season to get data
    how: str
        "full": statistics of teams when playing as home team and away team
        "home": statistics of teams when playing as home team
        "away": statistics of teams when playing as away team
    """

    full_week_results = []
    for week in range(1,39):
        full_week_results.append(team_stats(season,week,how))
        print(f'Got results of week {week} and season {season}')
    df = pd.concat(full_week_results).reset_index(drop=True)

    return df

def team_stats_full_season(how='full'):
    """
    Return some statistics of teams (number win matchs, number lose matchs, number draw matchs, rank, goal difference) in all week and all season
    
    Parameters:
    how: str
        "full": statistics of teams when playing as home team and away team
        "home": statistics of teams when playing as home team
        "away": statistics of teams when playing as away team
    """

    full_season_results = []
    for season in range(2001,2022):
        full_season_results.append(team_stats_full_week(season,how))
    df = pd.concat(full_season_results).reset_index(drop=True)

    return df

In [22]:
# stats_full = team_stats_full_season()
# stats_full.to_csv('data/team_stats_full.csv',index=False)

Got results of week 1 and season 2001
Got results of week 2 and season 2001
Got results of week 3 and season 2001
Got results of week 4 and season 2001
Got results of week 5 and season 2001
Got results of week 6 and season 2001
Got results of week 7 and season 2001
Got results of week 8 and season 2001
Got results of week 9 and season 2001
Got results of week 10 and season 2001
Got results of week 11 and season 2001
Got results of week 12 and season 2001
Got results of week 13 and season 2001
Got results of week 14 and season 2001
Got results of week 15 and season 2001
Got results of week 16 and season 2001
Got results of week 17 and season 2001
Got results of week 18 and season 2001
Got results of week 19 and season 2001
Got results of week 20 and season 2001
Got results of week 21 and season 2001
Got results of week 22 and season 2001
Got results of week 23 and season 2001
Got results of week 24 and season 2001
Got results of week 25 and season 2001
Got results of week 26 and season 

In [25]:
# stats_home = team_stats_full_season(how='home')
# stats_home.to_csv('data/team_stats_home.csv',index=False)

Got results of week 1 and season 2001
Got results of week 2 and season 2001
Got results of week 3 and season 2001
Got results of week 4 and season 2001
Got results of week 5 and season 2001
Got results of week 6 and season 2001
Got results of week 7 and season 2001
Got results of week 8 and season 2001
Got results of week 9 and season 2001
Got results of week 10 and season 2001
Got results of week 11 and season 2001
Got results of week 12 and season 2001
Got results of week 13 and season 2001
Got results of week 14 and season 2001
Got results of week 15 and season 2001
Got results of week 16 and season 2001
Got results of week 17 and season 2001
Got results of week 18 and season 2001
Got results of week 19 and season 2001
Got results of week 20 and season 2001
Got results of week 21 and season 2001
Got results of week 22 and season 2001
Got results of week 23 and season 2001
Got results of week 24 and season 2001
Got results of week 25 and season 2001
Got results of week 26 and season 

In [26]:
# stats_away = team_stats_full_season(how='away')
# stats_away.to_csv('data/team_stats_away.csv',index=False)

Got results of week 1 and season 2001
Got results of week 2 and season 2001
Got results of week 3 and season 2001
Got results of week 4 and season 2001
Got results of week 5 and season 2001
Got results of week 6 and season 2001
Got results of week 7 and season 2001
Got results of week 8 and season 2001
Got results of week 9 and season 2001
Got results of week 10 and season 2001
Got results of week 11 and season 2001
Got results of week 12 and season 2001
Got results of week 13 and season 2001
Got results of week 14 and season 2001
Got results of week 15 and season 2001
Got results of week 16 and season 2001
Got results of week 17 and season 2001
Got results of week 18 and season 2001
Got results of week 19 and season 2001
Got results of week 20 and season 2001
Got results of week 21 and season 2001
Got results of week 22 and season 2001
Got results of week 23 and season 2001
Got results of week 24 and season 2001
Got results of week 25 and season 2001
Got results of week 26 and season 

In [6]:
# season_2000_stats = team_stats_full_week(2000,how='full')
# season_2000_stats.to_csv('data/team_stats_2000.csv',index=False)
# season_2000_stats

Got results of week 1 and season 2000
Got results of week 2 and season 2000
Got results of week 3 and season 2000
Got results of week 4 and season 2000
Got results of week 5 and season 2000
Got results of week 6 and season 2000
Got results of week 7 and season 2000
Got results of week 8 and season 2000
Got results of week 9 and season 2000
Got results of week 10 and season 2000
Got results of week 11 and season 2000
Got results of week 12 and season 2000
Got results of week 13 and season 2000
Got results of week 14 and season 2000
Got results of week 15 and season 2000
Got results of week 16 and season 2000
Got results of week 17 and season 2000
Got results of week 18 and season 2000
Got results of week 19 and season 2000
Got results of week 20 and season 2000
Got results of week 21 and season 2000
Got results of week 22 and season 2000
Got results of week 23 and season 2000
Got results of week 24 and season 2000
Got results of week 25 and season 2000
Got results of week 26 and season 

,Team,W,D,L,Diff,Goal,Opponent Goal,Rank,wk,Season
0,Chelsea,1,0,0,4,4,0,1,1,2000
1,NaN,1,0,0,1,3,2,2,1,2000
2,Arsenal,1,0,0,1,2,1,3,1,2000
3,Liverpool,1,0,0,1,2,1,4,1,2000
4,Aston Villa,1,0,0,1,1,0,5,1,2000
...,...,...,...,...,...,...,...,...,...,...
755,Derby County,9,11,18,-13,44,57,16,38,2000
756,Bradford City,9,9,20,-30,38,68,17,38,2000
757,NaN,7,12,19,-28,46,74,18,38,2000
758,NaN,8,7,23,-32,38,70,19,38,2000


In [31]:
def team_values_11_21(season,when):
    """
    Return the values of teams of specified season (from season 2011 to 2021)

    Parameters:
    season: int
        Season to get data
    when: str
        "summer": after summer transfer window
        "winter": after winter transfer window
    """

    if when == 'summer':
        time = f'{season-1}-11-01'
        url = f'https://www.transfermarkt.com/premier-league/marktwerteverein/wettbewerb/GB1/plus/1?stichtag={time}'
    else:
        time = f'{season}-03-01'
        url = f'https://www.transfermarkt.com/premier-league/marktwerteverein/wettbewerb/GB1/plus/1?stichtag={time}'
    
    header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers=header).text
    soup = BeautifulSoup(r, 'html5lib')
    length_values_data = len(soup.find_all('td',class_='rechts'))
    data ={'Team':[],'Values':[]}
    for i in range(4,length_values_data,4):
        team_values = soup.find_all('td',class_='rechts')
        data['Values'].append(team_values[i].text)
    for i in soup.find_all('td',class_='hauptlink no-border-links'):
        data['Team'].append(i.text)
    df = pd.DataFrame(data)
    df['When'] = when
    df['Season'] = season
    
    return df

def team_values_11_21_full():
    """
    Return the values of teams from season 2011 to 2021
    """

    team_values = []
    for season in range(2011,2022):
        for time in ['summer','winter']:
            team_values.append(team_values_11_21(season,time))
            print(f'Done {season}-{time}')
    df = pd.concat(team_values).reset_index(drop=True)

    return df

In [34]:
def team_values_01_10(season):
    """
    Return the values of teams of specified season (from season 2001 to 2010)

    Parameters:
    season: int
        Season to get data
    """

    url = f'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/{season-1}'
    header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers=header).text
    soup = BeautifulSoup(r, 'html5lib')
    data ={'Team':[],'Values':[]}
    for i in range(3,42,2):
        team_values = soup.find_all('td',class_='rechts')
        data['Values'].append(team_values[i].text)
    for i in soup.find_all('td',class_='hauptlink no-border-links'):
        data['Team'].append(i.text.strip())
    
    df_summer = pd.DataFrame(data)
    df_summer['When'] = 'summer'
    df_summer['Season'] = season

    df_winter = df_summer.copy()
    df_winter['When'] = 'winter'

    df = pd.concat([df_summer,df_winter]).reset_index(drop=True)

    return df

def team_values_01_10_full():
    """
    Return the values of teams from season 2001 to 2010
    """

    team_values = []
    for season in range(2001,2011):
        team_values.append(team_values_01_10(season))
        print(f'Done {season}')
    df = pd.concat(team_values).reset_index(drop=True)

    return df

In [36]:
# values_01_10 = team_values_01_10_full()
# values_11_21 = team_values_11_21_full()
# values = [values_01_10, values_11_21]
# values = pd.concat(values)
# values.to_csv('data/team_values.csv',index=False)

Done 2001
Done 2002
Done 2003
Done 2004
Done 2005
Done 2006
Done 2007
Done 2008
Done 2009
Done 2010
Done 2011-summer
Done 2011-winter
Done 2012-summer
Done 2012-winter
Done 2013-summer
Done 2013-winter
Done 2014-summer
Done 2014-winter
Done 2015-summer
Done 2015-winter
Done 2016-summer
Done 2016-winter
Done 2017-summer
Done 2017-winter
Done 2018-summer
Done 2018-winter
Done 2019-summer
Done 2019-winter
Done 2020-summer
Done 2020-winter
Done 2021-summer
Done 2021-winter
